In [3]:
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.models import *
from keras.layers import *
from keras.applications import vgg19
import cv2
epoches = 50


def compute_content_cost(a_C, a_G):
    """
    Computes the content cost

    Arguments:
    a_C -- tensor of dimension (None, n_H, n_W, n_C), hidden layer activations representing content of the image C
    a_G -- tensor of dimension (None, n_H, n_W, n_C), hidden layer activations representing content of the image G

    Returns:
    J_content -- scalar that you compute using equation 1 above.
    """

    n_H, n_W, n_C = a_G.get_shape().as_list()
    a_C_unrolled = K.reshape(a_C, (-1, n_C))
    a_G_unrolled = K.reshape(a_G, (-1, n_C))

    # compute the cost with tensorflow (≈1 line)
    J_content = K.sum(K.square(a_C_unrolled - a_G_unrolled), axis=[0, 1]) / (4 * n_H * n_W * n_C)

    return J_content


def gram_matrix(A):
    """
    Argument:
    A -- matrix of shape (n_C, n_H*n_W)

    Returns:
    GA -- Gram matrix of A, of shape (n_C, n_C)
    """

    GA = K.dot(A, K.transpose(A))
    return GA


def compute_layer_style_cost(a_S, a_G):
    """
    Arguments:
    a_S -- tensor of dimension (None, n_H, n_W, n_C), hidden layer activations representing style of the image S
    a_G -- tensor of dimension (None, n_H, n_W, n_C), hidden layer activations representing style of the image G

    Returns:
    J_style_layer -- tensor representing a scalar value, style cost defined above by equation (2)
    """

    # Retrieve dimensions from a_G (≈1 line)
    n_H, n_W, n_C = a_G.get_shape().as_list()

    # Reshape the images to have them of shape (n_C, n_H*n_W) (≈2 lines)
    a_S = K.transpose(K.reshape(a_S, (-1, n_C)))
    a_G = K.transpose(K.reshape(a_G, (-1, n_C)))

    # Computing gram_matrices for both images S and G (≈2 lines)
    GS = gram_matrix(a_S)
    GG = gram_matrix(a_G)

    # Computing the loss (≈1 line)
    J_style_layer = K.sum(K.square(GS - GG), axis=[0, 1]) / (4 * n_C * n_C * n_H * n_H * n_W * n_W)

    return J_style_layer


content_image = cv2.imread('content.jpg')
content_image = np.expand_dims(content_image, axis=0)
style_image = cv2.imread('style.jpg')
style_image = np.expand_dims(style_image, axis=0)
m, n_H, n_W, n_C = content_image.shape

generator = Input(shape=(n_H, n_W, n_C))

content = K.variable(content_image)
style = K.variable(style_image)

input_tensor = Lambda(lambda x: K.concatenate([K.concatenate([content, style, x], axis=0)]))(generator)


vgg = vgg19.VGG19(weights='imagenet', include_top=False, input_tensor = input_tensor)
model = Model(generator, vgg.output)
model.summary()
# define style and content layers
STYLE_LAYERS = [
    ('block1_conv1', 0.2),
    ('block2_conv1', 0.2),
    ('block3_conv1', 0.2),
    ('block4_conv1', 0.2),
    ('block5_conv1', 0.2)]
content_layer_name = 'block5_conv2'

# content loss
content_layer = model.get_layer(content_layer_name).output
content_image_feature = content_layer[0]
generator_content_feature = content_layer[2]
content_loss = compute_content_cost(content_image_feature, generator_content_feature)

# style loss
style_loss = K.variable(0)
for i in STYLE_LAYERS:
    name, coeff = i
    style_layer = model.get_layer(name).output
    style_image_feature = style_layer[1]
    generator_style_feature = style_layer[2]
    style_loss = style_loss + compute_layer_style_cost(style_image_feature, generator_style_feature) * coeff

# total cost
loss = content_loss + style_loss

# gradients
grad = K.gradients(loss, generator)

# get gradient
delta = K.function([generator], [grad])

# iterate update
G = np.expand_dims(cv2.imread('content.jpg'), axis=0)
lr = 0.01
for i in range(epoches):
    update = delta(G)[0][0]
    update = lr * update

    G = G - update
    lr = lr * 0.99
    print(G)
    cv2.imwrite('merge' + str(i)+'.jpg', G[0])
    print('written epoche: ', i)













Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 300, 400, 3)       0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 300, 400, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 400, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 400, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 200, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 200, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 200, 128)     1475